# NIH awards data
https://exporter.nih.gov/ExPORTER_Catalog.aspx

In [1]:
import requests, zipfile, io
from bs4 import BeautifulSoup
import os

import pandas as pd
import numpy as np

import cleaning_strings as cln

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
url = 'https://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2016.zip'
r = requests.get(url).content
z = zipfile.ZipFile(io.BytesIO(r))
z.extractall()

csv = 'RePORTER_PRJ_C_FY2016.csv'
grants_2016_raw = pd.read_csv(csv, encoding = 'latin1')
os.remove(csv)

pd.set_option('display.max_columns', 50)
grants_2016_raw.head(10)

,APPLICATION_ID,ACTIVITY,ADMINISTERING_IC,APPLICATION_TYPE,ARRA_FUNDED,AWARD_NOTICE_DATE,BUDGET_START,BUDGET_END,CFDA_CODE,CORE_PROJECT_NUM,ED_INST_TYPE,FOA_NUMBER,FULL_PROJECT_NUM,FUNDING_ICs,FUNDING_MECHANISM,FY,IC_NAME,NIH_SPENDING_CATS,ORG_CITY,ORG_COUNTRY,ORG_DEPT,ORG_DISTRICT,ORG_DUNS,ORG_FIPS,ORG_NAME,ORG_STATE,ORG_ZIPCODE,PHR,PI_IDS,PI_NAMEs,PROGRAM_OFFICER_NAME,PROJECT_START,PROJECT_END,PROJECT_TERMS,PROJECT_TITLE,SERIAL_NUMBER,STUDY_SECTION,STUDY_SECTION_NAME,SUBPROJECT_ID,SUFFIX,SUPPORT_YEAR,DIRECT_COST_AMT,INDIRECT_COST_AMT,TOTAL_COST,TOTAL_COST_SUB_PROJECT
0,9115627,K23,GM,4.0,N,7/27/2016,8/1/2016,7/31/2017,859,K23GM104401,SCHOOLS OF MEDICINE,PA-11-009,4K23GM104401-04,NIGMS:194460\,OTHER RESEARCH-RELATED,2016,NATIONAL INSTITUTE OF GENERAL MEDICAL SCIENCES,Atherosclerosis; Biotechnology; Cardiovascular...,NEW YORK,UNITED STATES,GENETICS,13.0,78861598,US,ICAHN SCHOOL OF MEDICINE AT MOUNT SINAI,NY,100296574,PUBLIC HEALTH RELEVANCE: Antiplatelet response...,10799126;,"SCOTT, STUART ALEXANDER;","LONG, ROCHELLE M.",8/1/2013,7/31/2017,ABCB1 gene; Accounting; acute coronary syndrom...,The Pharmacogenomic Control of Clopidogrel Res...,104401.0,GHD,Genetics of Health and Disease Study Section,NaN,NaN,4.0,180500.0,13960.0,194460.0,NaN
1,9128072,R01,NS,4.0,N,8/15/2016,8/1/2016,7/31/2017,853,R01NS085165,SCHOOLS OF MEDICINE,PA-11-260,4R01NS085165-04,NINDS:335781\,Non-SBIR/STTR RPGs,2016,NATIONAL INSTITUTE OF NEUROLOGICAL DISORDERS A...,Biotechnology; Complementary and Alternative M...,BALTIMORE,UNITED STATES,ANESTHESIOLOGY,7.0,188435911,US,UNIVERSITY OF MARYLAND BALTIMORE,MD,212011508,PUBLIC HEALTH RELEVANCE: Activation of microgl...,7017365;,"POLSTER, BRIAN M;","MORRIS, JILL A",9/30/2013,7/31/2018,Acute; analog; Antioxidants; attenuation; Bind...,Novel Mechanisms of Microglial Neurotoxicity a...,85165.0,NOMD,Neural Oxidative Metabolism and Death Study Se...,NaN,NaN,4.0,218750.0,117031.0,335781.0,NaN
2,9056435,D43,TW,4.0,N,3/19/2016,3/1/2016,2/28/2017,989,D43TW009310,SCHOOLS OF MEDICINE,PAR-10-257,4D43TW009310-05,FIC:146822\NIDA:75000\,OTHER RESEARCH-RELATED,2016,FOGARTY INTERNATIONAL CENTER,Aging; Behavioral and Social Science; Brain Di...,ANN ARBOR,UNITED STATES,PSYCHIATRY,12.0,73133571,US,UNIVERSITY OF MICHIGAN,MI,481091276,Although recent science has documented the dev...,7354078; 1862210 (contact);,"WALTON, MAUREEN A; ZUCKER, ROBERT ALPERT (cont...","MICHELS, KATHLEEN M",7/9/2012,2/28/2017,21 year old; Academy; addiction; Address; Alco...,Capacity Building for Lifespan Focused Substan...,9310.0,ZRG1,Special Emphasis Panel,NaN,NaN,5.0,209478.0,12344.0,221822.0,NaN
3,9213716,R01,MH,1.0,N,9/8/2016,9/9/2016,5/31/2017,242,R01MH111502,SCHOOLS OF ARTS AND SCIENCES,RFA-MH-16-775,1R01MH111502-01,NIMH:1593922\,Non-SBIR/STTR RPGs,2016,NATIONAL INSTITUTE OF MENTAL HEALTH,Basic Behavioral and Social Science; Behaviora...,CAMBRIDGE,UNITED STATES,MICROBIOLOGY/IMMUN/VIROLOGY,5.0,82359691,US,HARVARD UNIVERSITY,MA,21385369,Project Narrative To shed light into the basic...,2275890 (contact); 8742217; 6139020;,"DULAC, CATHERINE G (contact); REGEV, AVIV ; ZH...","FREUND, MICHELLE",9/9/2016,5/31/2019,abstracting; Accounting; Address; Aggressive b...,In situ transcriptional analysis of brain circ...,111502.0,ZMH1,Special Emphasis Panel,NaN,NaN,1.0,1077309.0,516613.0,1593922.0,NaN
4,8986215,R01,MH,5.0,N,11/5/2015,12/1/2015,11/30/2016,242,R01MH105329,SCHOOLS OF MEDICINE,PA-14-094,5R01MH105329-02,NIMH:294755\,Non-SBIR/STTR RPGs,2016,NATIONAL INSTITUTE OF MENTAL HEALTH,Acquired Cognitive Impairment; Brain Disorders...,GRAND FORKS,UNITED STATES,OTHER BASIC SCIENCES,0.0,102280781,US,UNIVERSITY OF NORTH DAKOTA,ND,582026059,PUBLIC HEALTH RELEVANCE: The proposed research...,9868481; 2063694 (contact);,"CHEN, XUESONG ; GEIGER, JONATHAN DAVID (contact);","JOSEPH, JEYMOHAN",12/15/2014,11/30/2019,Affect; Anti-Retroviral Agents; antiretroviral...,Role of novel endolysosome-dependent calcium r...,105329.0,ZRG1,Special Emphasis Panel,NaN,NaN,2.0,212500.0,82255.

In [3]:
grants_2016_raw.shape

(71872, 45)

# Processing data

## Basic text processing of column names

In [230]:
grants_2016 = grants_2016_raw.copy()

grants_2016.columns = grants_2016.columns.str.lower()
grants_2016.head(1)

,application_id,activity,administering_ic,application_type,arra_funded,award_notice_date,budget_start,budget_end,cfda_code,core_project_num,ed_inst_type,foa_number,full_project_num,funding_ics,funding_mechanism,fy,ic_name,nih_spending_cats,org_city,org_country,org_dept,org_district,org_duns,org_fips,org_name,org_state,org_zipcode,phr,pi_ids,pi_names,program_officer_name,project_start,project_end,project_terms,project_title,serial_number,study_section,study_section_name,subproject_id,suffix,support_year,direct_cost_amt,indirect_cost_amt,total_cost,total_cost_sub_project
0,9115627,K23,GM,4.0,N,7/27/2016,8/1/2016,7/31/2017,859,K23GM104401,SCHOOLS OF MEDICINE,PA-11-009,4K23GM104401-04,NIGMS:194460\,OTHER RESEARCH-RELATED,2016,NATIONAL INSTITUTE OF GENERAL MEDICAL SCIENCES,Atherosclerosis; Biotechnology; Cardiovascular...,NEW YORK,UNITED STATES,GENETICS,13.0,78861598,US,ICAHN SCHOOL OF MEDICINE AT MOUNT SINAI,NY,100296574,PUBLIC HEALTH RELEVANCE: Antiplatelet response...,10799126;,"SCOTT, STUART ALEXANDER;","LONG, ROCHELLE M.",8/1/2013,7/31/2017,ABCB1 gene; Accounting; acute coronary syndrom...,The Pharmacogenomic Control of Clopidogrel Res...,104401.0,GHD,Genetics of Health and Disease Study Section,NaN,NaN,4.0,180500.0,13960.0,194460.0,NaN


In [231]:
#converts df to lowercase string
grants_2016 = grants_2016.apply(lambda x: x.astype(str).str.lower())
grants_2016.head(1)

,application_id,activity,administering_ic,application_type,arra_funded,award_notice_date,budget_start,budget_end,cfda_code,core_project_num,ed_inst_type,foa_number,full_project_num,funding_ics,funding_mechanism,fy,ic_name,nih_spending_cats,org_city,org_country,org_dept,org_district,org_duns,org_fips,org_name,org_state,org_zipcode,phr,pi_ids,pi_names,program_officer_name,project_start,project_end,project_terms,project_title,serial_number,study_section,study_section_name,subproject_id,suffix,support_year,direct_cost_amt,indirect_cost_amt,total_cost,total_cost_sub_project
0,9115627,k23,gm,4.0,n,7/27/2016,8/1/2016,7/31/2017,859,k23gm104401,schools of medicine,pa-11-009,4k23gm104401-04,nigms:194460\,other research-related,2016,national institute of general medical sciences,atherosclerosis; biotechnology; cardiovascular...,new york,united states,genetics,13.0,78861598,us,icahn school of medicine at mount sinai,ny,100296574,public health relevance: antiplatelet response...,10799126;,"scott, stuart alexander;","long, rochelle m.",8/1/2013,7/31/2017,abcb1 gene; accounting; acute coronary syndrom...,the pharmacogenomic control of clopidogrel res...,104401.0,ghd,genetics of health and disease study section,nan,nan,4.0,180500.0,13960.0,194460.0,nan


In [232]:
#converts 'nan' to NaN
grants_2016.replace(['nan'], np.nan, inplace=True)
grants_2016.head(1)

,application_id,activity,administering_ic,application_type,arra_funded,award_notice_date,budget_start,budget_end,cfda_code,core_project_num,ed_inst_type,foa_number,full_project_num,funding_ics,funding_mechanism,fy,ic_name,nih_spending_cats,org_city,org_country,org_dept,org_district,org_duns,org_fips,org_name,org_state,org_zipcode,phr,pi_ids,pi_names,program_officer_name,project_start,project_end,project_terms,project_title,serial_number,study_section,study_section_name,subproject_id,suffix,support_year,direct_cost_amt,indirect_cost_amt,total_cost,total_cost_sub_project
0,9115627,k23,gm,4.0,n,7/27/2016,8/1/2016,7/31/2017,859,k23gm104401,schools of medicine,pa-11-009,4k23gm104401-04,nigms:194460\,other research-related,2016,national institute of general medical sciences,atherosclerosis; biotechnology; cardiovascular...,new york,united states,genetics,13.0,78861598,us,icahn school of medicine at mount sinai,ny,100296574,public health relevance: antiplatelet response...,10799126;,"scott, stuart alexander;","long, rochelle m.",8/1/2013,7/31/2017,abcb1 gene; accounting; acute coronary syndrom...,the pharmacogenomic control of clopidogrel res...,104401.0,ghd,genetics of health and disease study section,NaN,NaN,4.0,180500.0,13960.0,194460.0,NaN


In [233]:
#Use this csv to clean PI information
grants_2016.to_csv('grants_all.csv', compression = 'gzip')

## Selecting columns for analysis
There are a total of 45 columns, which may contain redundant or unnecessary information.

In [234]:
pd.set_option('display.max_colwidth', 5000)
col_info = pd.read_csv('grant_col_info_all.csv')
col_info[:9]

,column_name,descriptions
0,application_id,A unique identifier of the project record in the ExPORTER database.
1,activity,"A 3-character code identifying the grant, contract, or intramural activity through which a project is supported. Within each funding mechanism , NIH uses 3-character activity codes (e.g., F32, K08, P01, R01, T32, etc.) to differentiate the wide variety of research-related programs NIH supports. A comprehensive list of activity codes for grants and cooperative agreements may be found on the Types of Grant Programs Web page. RePORTER also includes R&D contracts (activity codes beginning with the letter N) and intramural projects (beginning with the letter Z)."
2,administering_ic,"Administering Institute or Center - A two-character code to designate the agency,NIH Institute, or Center administering the grant. See Institute/Center code definitions"
3,application_type,"A one-digit code to identify the type of application funded: 1 = New application 2 = Competing continuation (also, competing renewal) 3 = Application for additional (supplemental) support. There are two kinds of type 3competing revisions (which are peer-reviewed and administrative supplements) 4 = Competing extension for an R37 award or first non-competing year of a Fast Track SBIR/STTR award 5 = Non-competing continuation 7 = Change of grantee institution 9 = Change of NIH awarding Institute or Division (on a competing continuation)"
4,arra_funded,“Y” indicates a project supported by funds appropriated through the American Recovery and Reinvestment Act of 2009.
5,award_notice_date,Award notice date or Notice of Grant Award (NGA) is a legally binding document stating the government has obligated funds and which defines the period of support and the terms and conditions of award.\r\n
6,budget_start,The date when a project’s funding for a particular fiscal year begins.
7,budget_end,The date when a project’s funding for a particular fiscal year ends.
8,cfda_code,"Federal programs are assigned a number in the Catalog of Federal Domestic Assistance (CFDA), which is referred to as the ""CFDA code."" The CFDA database helps the Federal government track all programs it has domestically funded. \r\n"


To remove (initial): 5-8, redundant information

In [235]:
to_drop = ['award_notice_date', 'budget_start', 'budget_end', 'cfda_code']
grants_2016 = grants_2016.drop(to_drop, axis = 1)

In [236]:
col_info[9:18]

,column_name,descriptions
9,core_project_num,"An identifier for each research project, used to associate the project with publication and patent records. This identifier is not specific to any particular year of the project. It consists of the project activity code, administering IC, and serial number (a concatenation of Activity, Administering_IC, and Serial_Number). \r\n"
10,ed_inst_type,Generic name for the grouping of components across an institution who has applied for or receives NIH funding. The official name as used by NIH is Major Component Combining Name. \r\n
11,foa_number,"The number of the funding opportunity announcement, if any, under which the project application was solicited. Funding opportunity announcements may be categorized as program announcements, requests for applications, notices of funding availability, solicitations, or other names depending on the agency and type of program. Funding opportunity announcements can be found at Grants.gov/FIND and in the NIH Guide for Grants and Contracts"
12,full_project_num,"Commonly referred to as a grant number, intramural project, or contract number. For grants, this unique identification number is composed of the type code, activity code, Institute/Center code, serial number, support year, and (optional) a suffix code to designate amended applications and supplements."
13,funding_ic(s),"The NIH Institute or Center(s) providing funding for a project are designated by their acronyms (see Institute/Center acronyms ). Each funding IC is followed by a colon (:) and the amount of funding provided for the fiscal year by that IC. Multiple ICs are separated by semicolons (;). Project funding information is available only for NIH, CDC, and FDA projects ."
14,funding_mechanism,"The major mechanism categories used in NIH Budget mechanism tables for the President’s budget. Extramural research awards are divided into three main funding mechanisms: grants, cooperative agreements and contracts. A funding mechanism is the type of funded application or transaction used at the NIH. Within each funding mechanism NIH includes programs. Programs can be further refined by specific activity codes."
15,fy,The fiscal year appropriation from which project funds were obligated.
16,ic_name,"Full name of the administering agency, Institute, or Center."
17,nih_spending_cats,"Congressionally-mandated reporting categories into which NIH projects are categorized. Available for fiscal years 2008 and later. Each project’s spending category designations for each fiscal year are made available the following year as part of the next President’s Budget request. See the Research, Condition, and Disease Categorization System for more information on the categorization process."


To remove: 9 (may add in later if correlating with publications), 10, 11, 16; also 2 (redundant with funding_ics)

In [237]:
to_drop2 = ['administering_ic', 'core_project_num', 'ed_inst_type', 'foa_number', 'ic_name']
grants_2016 = grants_2016.drop(to_drop2, axis = 1)

In [238]:
col_info[18:27]

,column_name,descriptions
18,org_city,"The city in which the business office of the grantee organization or contractor is located. Note that this may be different from the research performance site. For all NIH intramural projects, Bethesda, MD is used."
19,org_country,The country in which the business office of the grantee organization or contractor is located. Note that this may be different from the research performance site.
20,org_dept,"The departmental affiliation of the contact principal investigator for a project, using a standardized categorization of departments. Names are available only for medical school departments."
21,org_district,The congressional district in which the business office of the grantee organization or contractor is located. Note that this may be different from the research performance site.
22,org_duns,"This field may contain multiple DUNS Numbers separated by a semi-colon. The Data Universal Numbering System is a unique nine-digit number assigned by Dun and Bradstreet Information Services, recognized as the universal standard for identifying and keeping track of business worldwide. \r\n"
23,org_fips,The country code of the grantee organization or contractor as defined in the Federal Information Processing Standard.
24,org_name,"The name of the educational institution, research organization, business, or government agency receiving funding for the grant, contract, cooperative agreement, or intramural project."
25,org_state,The state in which the business office of the grantee organization or contractor is located. Note that this may be different from the research performance site.
26,org_zipcode,The zip code in which the business office of the grantee organization or contractor is located. Note that this may be different from the research performance site.


All redundant: 18, 20-23, 25

Also remove 19, 24, 26; these will be added again later.

In [239]:
to_drop3 = ['org_city', 'org_country', 'org_dept', 'org_district', 'org_duns', 'org_fips', 'org_name', 'org_state', 'org_zipcode']
grants_2016 = grants_2016.drop(to_drop3, axis = 1)

In [240]:
col_info[27:36]

,column_name,descriptions
27,phr,"Submitted as part of a grant application, this statement articulates a project's potential to improve public health."
28,pi_id(s),A unique identifier for each of the project Principal Investigators. Each PI in the RePORTER database has a unique identifier that is constant from project to project and year to year.
29,pi_name(s),The name(s) of the Principal Investigator(s) designated by the organization to direct the research project.
30,program_officer_name,An Institute staff member who coordinates the substantive aspects of a contract from planning the request for proposal to oversight.
31,project_start,"The start date of a project. For subprojects of a multi-project grant, this is the start date of the parent award."
32,project_end,"The current end date of the project, including any future years for which commitments have been made. For subprojects of a multi-project grant, this is the end date of the parent award. Upon competitive renewal of a grant, the project end date is extended by the length of the renewal award."
33,project_terms,"Prior to fiscal year 2008, these were thesaurus terms assigned by NIH CRISP indexers. For projects funded in fiscal year 2008 and later, these are concepts that are mined from the project's title, abstract, and specific aims using an automated text mining tool."
34,project_title,"Title of the funded grant, contract, or intramural (sub)project."
35,serial_number,A six-digit number assigned in serial number order within each administering organization.


27 (phr): maybe later for text analysis, too complex for initial analysis; 29, 34, 35

In [241]:
to_drop4 = ['phr', 'pi_names', 'program_officer_name', 'project_title', 'serial_number']
grants_2016 = grants_2016.drop(to_drop4, axis = 1)

In [242]:
col_info[36:]

,column_name,descriptions
36,study_section,A designator of the legislatively-mandated panel of subject matter experts that reviewed the research grant application for scientific and technical merit.
37,study_section_name,The full name of a regular standing Study Section that reviewed the research grant application for scientific and technical merit. Applications reviewed by panels other than regular standing study sections are designated by “Special Emphasis Panel.”
38,subproject_id,A unique numeric designation assigned to subprojects of a “parent” multi-project research grant.
39,suffix,"A suffix to the grant application number that includes the letter ""A"" and a serial number to identify an amended version of an original application and/or the letter ""S"" and serial number indicating a supplement to the project. ."
40,support_year,"The year of support for a project, as shown in the full project number. For example, a project with number 5R01GM0123456-04 is in its fourth year of support."
41,direct_cost_amt,Total indirect cost funding for a project from all NIH Institute and Centers for a given fiscal year. Costs are available only for NIH awards funded in FY 2012 and onward. Indirect cost amounts are not available for SBIR/STTR awards.
42,indirect_cost_amt,Total indirect cost funding for a project from all NIH Institute and Centers for a given fiscal year. Costs are available only for NIH awards funded in FY 2012 and onward. Indirect cost amounts are not available for SBIR/STTR awards.
43,total_cost,"Total project funding from all NIH Institute and Centers for a given fiscal year. Costs are available only for: NIH, CDC, and FDA grant awards (only the parent record of multi-project grants). -NIH intramural projects (activity codes beginning with “Z”) in FY 2007 and later fiscal years. -NIH contracts (activity codes beginning with “N”) . For multi-project grants, Total_Cost includes funding for all of the constituent subprojects. This field will be blank on subproject records; the total cost of each subproject is found in Total_Cost_Sub_Project ."
44,total_cost_sub_project,Applies to subproject records only. Total funding for a subproject from all NIH Institute and Centers for a given fiscal year. Costs are available only for NIH awards.


37 (redundant with 36, can always look up)

In [245]:
#check number of subprojects
#grants_2016.shape
#subproject_cols = ['subproject_id', 'suffix', 'total_cost_sub_project']
#grants_2016[subproject_cols].isnull().sum()

In [246]:
grants_2016 = grants_2016.drop(['study_section_name', 'subproject_id', 'suffix'], axis = 1)

In [247]:
pd.set_option('display.max_colwidth', 50)
grants_2016.head()

,application_id,activity,application_type,arra_funded,full_project_num,funding_ics,funding_mechanism,fy,nih_spending_cats,pi_ids,project_start,project_end,project_terms,study_section,support_year,direct_cost_amt,indirect_cost_amt,total_cost,total_cost_sub_project
0,9115627,k23,4.0,n,4k23gm104401-04,nigms:194460\,other research-related,2016,atherosclerosis; biotechnology; cardiovascular...,10799126;,8/1/2013,7/31/2017,abcb1 gene; accounting; acute coronary syndrom...,ghd,4.0,180500.0,13960.0,194460.0,NaN
1,9128072,r01,4.0,n,4r01ns085165-04,ninds:335781\,non-sbir/sttr rpgs,2016,biotechnology; complementary and alternative m...,7017365;,9/30/2013,7/31/2018,acute; analog; antioxidants; attenuation; bind...,nomd,4.0,218750.0,117031.0,335781.0,NaN
2,9056435,d43,4.0,n,4d43tw009310-05,fic:146822\nida:75000\,other research-related,2016,aging; behavioral and social science; brain di...,7354078; 1862210 (contact);,7/9/2012,2/28/2017,21 year old; academy; addiction; address; alco...,zrg1,5.0,209478.0,12344.0,221822.0,NaN
3,9213716,r01,1.0,n,1r01mh111502-01,nimh:1593922\,non-sbir/sttr rpgs,2016,basic behavioral and social science; behaviora...,2275890 (contact); 8742217; 6139020;,9/9/2016,5/31/2019,abstracting; accounting; address; aggressive b...,zmh1,1.0,1077309.0,516613.0,1593922.0,NaN
4,8986215,r01,5.0,n,5r01mh105329-02,nimh:294755\,non-sbir/sttr rpgs,2016,acquired cognitive impairment; brain disorders...,9868481; 2063694 (contact);,12/15/2014,11/30/2019,affect; anti-retroviral agents; antiretroviral...,zrg1,2.0,212500.0,82255.0,294755.0,NaN


### More cleaning of columns

Convert strings to numeric and dates

In [248]:
def convert_column(df, list_of_names, function):
    '''
    Converts df column values from x to 'function'.
    Function = numeric: pd.to_numeric()
    Function = date: pd.to_datetime()
    Takes a df and a list with the column names.
    Returns df.
    '''
    for name in list_of_names:
        if function == 'numeric':
            df[name] = pd.to_numeric(df[name])
        elif function == 'date':
            df[name] = pd.to_datetime(df[name], errors = 'coerce')
    return df

numeric_cols = ['direct_cost_amt', 'indirect_cost_amt', 'total_cost', 'total_cost_sub_project']
grants_2016 = convert_column(grants_2016, numeric_cols, function = 'numeric')

date_cols = ['project_start', 'project_end']
grants_2016 = convert_column(grants_2016, date_cols, function = 'date')


grants_2016.head(1)

,application_id,activity,application_type,arra_funded,full_project_num,funding_ics,funding_mechanism,fy,nih_spending_cats,pi_ids,project_start,project_end,project_terms,study_section,support_year,direct_cost_amt,indirect_cost_amt,total_cost,total_cost_sub_project
0,9115627,k23,4.0,n,4k23gm104401-04,nigms:194460\,other research-related,2016,atherosclerosis; biotechnology; cardiovascular...,10799126;,2013-08-01,2017-07-31,abcb1 gene; accounting; acute coronary syndrom...,ghd,4.0,180500.0,13960.0,194460.0,NaN


## Splitting individual PIs when more than one is listed on a grant
Split grants with multiple PIs so that each row only has a single PI listed. This will allow analysis on funding per individual and per institution.

In [249]:
col_list = ['nih_spending_cats', 'pi_ids', 'project_terms']
grants_2016 = cln.strip_series(grants_2016, col_list, strip = '; ')
grants_2016 = cln.split_rows_pis(grants_2016, col_name = 'pi_ids')

More cleaning

In [250]:
grants_2016 = cln.strip_series(grants_2016, ['pi_ids'])
grants_2016 = cln.strip_series(grants_2016, ['pi_ids'], strip = ' ')
grants_2016.head()

,application_id,activity,application_type,arra_funded,full_project_num,funding_ics,funding_mechanism,fy,nih_spending_cats,project_start,project_end,project_terms,study_section,support_year,direct_cost_amt,indirect_cost_amt,total_cost,total_cost_sub_project,pi_ids
0,9115627,k23,4.0,n,4k23gm104401-04,nigms:194460\,other research-related,2016,atherosclerosis; biotechnology; cardiovascular...,2013-08-01,2017-07-31,abcb1 gene; accounting; acute coronary syndrom...,ghd,4.0,180500.0,13960.0,194460.0,NaN,10799126
1,9128072,r01,4.0,n,4r01ns085165-04,ninds:335781\,non-sbir/sttr rpgs,2016,biotechnology; complementary and alternative m...,2013-09-30,2018-07-31,acute; analog; antioxidants; attenuation; bind...,nomd,4.0,218750.0,117031.0,335781.0,NaN,7017365
2,9056435,d43,4.0,n,4d43tw009310-05,fic:146822\nida:75000\,other research-related,2016,aging; behavioral and social science; brain di...,2012-07-09,2017-02-28,21 year old; academy; addiction; address; alco...,zrg1,5.0,209478.0,12344.0,221822.0,NaN,7354078
2,9056435,d43,4.0,n,4d43tw009310-05,fic:146822\nida:75000\,other research-related,2016,aging; behavioral and social science; brain di...,2012-07-09,2017-02-28,21 year old; academy; addiction; address; alco...,zrg1,5.0,209478.0,12344.0,221822.0,NaN,1862210
3,9213716,r01,1.0,n,1r01mh111502-01,nimh:1593922\,non-sbir/sttr rpgs,2016,basic behavioral and social science; behaviora...,2016-09-09,2019-05-31,abstracting; accounting; address; aggressive b...,zmh1,1.0,1077309.0,516613.0,1593922.0,NaN,2275890


Split grant totals by number of associated PIs (assumption is that all PIs on a grant receive the same amount of money).

In [251]:
#divide cost columns by # of times the application_id occurs
pi_per_grant = pd.DataFrame(grants_2016['application_id'].value_counts())
pi_per_grant = pi_per_grant.reset_index()
pi_per_grant.columns = ['application_id', 'num_pis']

#Match application IDs in grants_2016 and pi_per_grant
#Divide the 3 cost columns by 
grants_2016 = pd.merge(grants_2016, pi_per_grant, on = 'application_id')

In [252]:
grants_2016.head()

,application_id,activity,application_type,arra_funded,full_project_num,funding_ics,funding_mechanism,fy,nih_spending_cats,project_start,project_end,project_terms,study_section,support_year,direct_cost_amt,indirect_cost_amt,total_cost,total_cost_sub_project,pi_ids,num_pis
0,9115627,k23,4.0,n,4k23gm104401-04,nigms:194460\,other research-related,2016,atherosclerosis; biotechnology; cardiovascular...,2013-08-01,2017-07-31,abcb1 gene; accounting; acute coronary syndrom...,ghd,4.0,180500.0,13960.0,194460.0,NaN,10799126,1
1,9128072,r01,4.0,n,4r01ns085165-04,ninds:335781\,non-sbir/sttr rpgs,2016,biotechnology; complementary and alternative m...,2013-09-30,2018-07-31,acute; analog; antioxidants; attenuation; bind...,nomd,4.0,218750.0,117031.0,335781.0,NaN,7017365,1
2,9056435,d43,4.0,n,4d43tw009310-05,fic:146822\nida:75000\,other research-related,2016,aging; behavioral and social science; brain di...,2012-07-09,2017-02-28,21 year old; academy; addiction; address; alco...,zrg1,5.0,209478.0,12344.0,221822.0,NaN,7354078,2
3,9056435,d43,4.0,n,4d43tw009310-05,fic:146822\nida:75000\,other research-related,2016,aging; behavioral and social science; brain di...,2012-07-09,2017-02-28,21 year old; academy; addiction; address; alco...,zrg1,5.0,209478.0,12344.0,221822.0,NaN,1862210,2
4,9213716,r01,1.0,n,1r01mh111502-01,nimh:1593922\,non-sbir/sttr rpgs,2016,basic behavioral and social science; behaviora...,2016-09-09,2019-05-31,abstracting; accounting; address; aggressive b...,zmh1,1.0,1077309.0,516613.0,1593922.0,NaN,2275890,3


In [253]:
def divide_by_column(df, col_list = ['direct_cost_amt', 'indirect_cost_amt', 'total_cost'], divide_by = 'num_pis'):
    '''
    
    Divide df columns by value in another column
    
    '''
    for col in col_list:
        df[col] = round(df[col] / df[divide_by])
    return df

grants_2016 = divide_by_column(grants_2016)

In [254]:
grants_2016.head()

,application_id,activity,application_type,arra_funded,full_project_num,funding_ics,funding_mechanism,fy,nih_spending_cats,project_start,project_end,project_terms,study_section,support_year,direct_cost_amt,indirect_cost_amt,total_cost,total_cost_sub_project,pi_ids,num_pis
0,9115627,k23,4.0,n,4k23gm104401-04,nigms:194460\,other research-related,2016,atherosclerosis; biotechnology; cardiovascular...,2013-08-01,2017-07-31,abcb1 gene; accounting; acute coronary syndrom...,ghd,4.0,180500.0,13960.0,194460.0,NaN,10799126,1
1,9128072,r01,4.0,n,4r01ns085165-04,ninds:335781\,non-sbir/sttr rpgs,2016,biotechnology; complementary and alternative m...,2013-09-30,2018-07-31,acute; analog; antioxidants; attenuation; bind...,nomd,4.0,218750.0,117031.0,335781.0,NaN,7017365,1
2,9056435,d43,4.0,n,4d43tw009310-05,fic:146822\nida:75000\,other research-related,2016,aging; behavioral and social science; brain di...,2012-07-09,2017-02-28,21 year old; academy; addiction; address; alco...,zrg1,5.0,104739.0,6172.0,110911.0,NaN,7354078,2
3,9056435,d43,4.0,n,4d43tw009310-05,fic:146822\nida:75000\,other research-related,2016,aging; behavioral and social science; brain di...,2012-07-09,2017-02-28,21 year old; academy; addiction; address; alco...,zrg1,5.0,104739.0,6172.0,110911.0,NaN,1862210,2
4,9213716,r01,1.0,n,1r01mh111502-01,nimh:1593922\,non-sbir/sttr rpgs,2016,basic behavioral and social science; behaviora...,2016-09-09,2019-05-31,abstracting; accounting; address; aggressive b...,zmh1,1.0,359103.0,172204.0,531307.0,NaN,2275890,3


# Testing merging of information

## Add organization information

Import pi_info.csv, which contains the necessary information.

In [255]:
cols_import = ['pi_ids', 'org_name', 'org_country', 'org_zipcode']
pi_info = pd.read_csv('pi_info.csv', compression = 'gzip', usecols = cols_import, dtype = {'pi_ids':str})
pi_info.head()

,pi_ids,org_name,org_country,org_zipcode
0,10799126,icahn school of medicine at mount sinai,united states,100296574
1,7017365,university of maryland baltimore,united states,212011508
2,1862210,university of michigan,united states,481091276
3,1866930,massachusetts institute of technology,united states,021421029
4,9851446,cold spring harbor laboratory,united states,117242209


Merge the two dataframes so the information is listed together.

In [256]:
grants_2016 = pd.merge(grants_2016, pi_info, left_on = 'pi_ids', right_on = 'pi_ids', how = 'left')

In [257]:
grants_2016.head(10)

,application_id,activity,application_type,arra_funded,full_project_num,funding_ics,funding_mechanism,fy,nih_spending_cats,project_start,project_end,project_terms,study_section,support_year,direct_cost_amt,indirect_cost_amt,total_cost,total_cost_sub_project,pi_ids,num_pis,org_name,org_country,org_zipcode
0,9115627,k23,4.0,n,4k23gm104401-04,nigms:194460\,other research-related,2016,atherosclerosis; biotechnology; cardiovascular...,2013-08-01,2017-07-31,abcb1 gene; accounting; acute coronary syndrom...,ghd,4.0,180500.0,13960.0,194460.0,NaN,10799126,1,icahn school of medicine at mount sinai,united states,100296574
1,9128072,r01,4.0,n,4r01ns085165-04,ninds:335781\,non-sbir/sttr rpgs,2016,biotechnology; complementary and alternative m...,2013-09-30,2018-07-31,acute; analog; antioxidants; attenuation; bind...,nomd,4.0,218750.0,117031.0,335781.0,NaN,7017365,1,university of maryland baltimore,united states,212011508
2,9056435,d43,4.0,n,4d43tw009310-05,fic:146822\nida:75000\,other research-related,2016,aging; behavioral and social science; brain di...,2012-07-09,2017-02-28,21 year old; academy; addiction; address; alco...,zrg1,5.0,104739.0,6172.0,110911.0,NaN,7354078,2,university of michigan,united states,481091276
3,9056435,d43,4.0,n,4d43tw009310-05,fic:146822\nida:75000\,other research-related,2016,aging; behavioral and social science; brain di...,2012-07-09,2017-02-28,21 year old; academy; addiction; address; alco...,zrg1,5.0,104739.0,6172.0,110911.0,NaN,1862210,2,university of michigan,united states,481091276
4,9213716,r01,1.0,n,1r01mh111502-01,nimh:1593922\,non-sbir/sttr rpgs,2016,basic behavioral and social science; behaviora...,2016-09-09,2019-05-31,abstracting; accounting; address; aggressive b...,zmh1,1.0,359103.0,172204.0,531307.0,NaN,2275890,3,harvard university,united states,021385369
5,9213716,r01,1.0,n,1r01mh111502-01,nimh:1593922\,non-sbir/sttr rpgs,2016,basic behavioral and social science; behaviora...,2016-09-09,2019-05-31,abstracting; accounting; address; aggressive b...,zmh1,1.0,359103.0,172204.0,531307.0,NaN,8742217,3,"broad institute, inc",united states,021421401
6,9213716,r01,1.0,n,1r01mh111502-01,nimh:1593922\,non-sbir/sttr rpgs,2016,basic behavioral and social science; behaviora...,2016-09-09,2019-05-31,abstracting; accounting; address; aggressive b...,zmh1,1.0,359103.0,172204.0,531307.0,NaN,6139020,3,harvard university,united states,021385369
7,8986215,r01,5.0,n,5r01mh105329-02,nimh:294755\,non-sbir/sttr rpgs,2016,acquired cognitive impairment; brain disorders...,2014-12-15,2019-11-30,affect; anti-retroviral agents; antiretroviral...,zrg1,2.0,106250.0,41128.0,147378.0,NaN,9868481,2,university of north dakota,united states,582026059
8,8986215,r01,5.0,n,5r01mh105329-02,nimh:294755\,non-sbir/sttr rpgs,2016,acquired cognitive impairment; brain disorders...,2014-12-15,2019-11-30,affect; anti-retroviral agents; antiretroviral...,zrg1,2.0,106250.0,41128.0,147378.0,NaN,2063694,2,university of north dakota,united states,582026059
9,9090084,p41,4.0,n,4p41eb015871-30,nibib:992896\,research centers,2016,bioengineering; biotechnology; cancer; regener...,1997-06-01,2017-05-31,area; automobile driving; biological research;...,zrg1,30.0,630065.0,362831.0,992896.0,NaN,1866930,1,massachusetts institute of technology,united states,021421029


In [258]:
not_null.head(10)

,application_id,activity,application_type,arra_funded,full_project_num,funding_ics,funding_mechanism,fy,nih_spending_cats,project_start,project_end,project_terms,study_section,support_year,direct_cost_amt,indirect_cost_amt,total_cost,total_cost_sub_project,pi_ids,num_pis,org_name,org_country,org_zipcode
19,9359909,zia,1.0,n,1ziatr000230-02,ncats:226982\,intramural research,2016,biotechnology; cancer,NaT,NaT,analog; antineoplastic agents; assay developme...,NaN,2.0,NaN,NaN,226982.0,NaN,14380202,1,translational science,united states,20892
20,9359855,zia,1.0,n,1zialm010001-07,nlm:530704\,intramural research,2016,networking and information technology r&d,NaT,NaT,abstracting; academia; anatomy; base; bioimagi...,NaN,7.0,NaN,NaN,530704.0,NaN,10687426,1,national library of medicine,united states,20892
21,9351937,zia,1.0,n,1ziaag000262-09,nia:202174\,intramural research,2016,aging; assistive technology; bioengineering; c...,NaT,NaT,adult; age; age difference; age related; aged;...,NaN,9.0,NaN,NaN,202174.0,NaN,9691085,1,aging,united states,20892
27,9352075,zia,1.0,n,1ziadc000064-16,nidcd:891718\,intramural research,2016,clinical research; congenital structural anoma...,NaT,NaT,acoustic nerve; action potentials; adolescent;...,NaN,16.0,NaN,NaN,891718.0,NaN,10271451,1,deafness & other communication disorders,united states,20892
35,9146677,p01,5.0,n,5p01ns069539-07,ninds:217794\,non-sbir/sttr rpgs,2016,facioscapulohumeral muscular dystrophy; geneti...,NaT,NaT,abstracting; affect; breeding; cells; chromati...,nsd,7.0,207242.0,10552.0,NaN,217794.0,6828192,1,fred hutchinson cancer research center,united states,981094433
36,9074409,p01,1.0,n,1p01ca203657-01,nci:151970\,non-sbir/sttr rpgs,2016,cancer; digestive diseases; genetics; orphan d...,2016-06-22,2021-05-31,addiction; address; alleles; automobile drivin...,zca1,1.0,151970.0,NaN,NaN,151970.0,1886390,1,univ of north carolina chapel hill,united states,275990001
37,9058513,p50,4.0,n,4p50da033935-04,nida:258794\,research centers,2016,basic behavioral and social science; behaviora...,NaT,NaT,achievement; addiction; agonist; behavior; beh...,zda1,4.0,258794.0,NaN,NaN,258794.0,1897523,1,university of texas medical br galveston,united states,775555302
38,9114012,u19,5.0,n,5u19ai113048-03,niaid:779012\,non-sbir/sttr rpgs,2016,clinical research; clinical trials and support...,NaT,NaT,accounting; adherence; adverse event; affect; ...,zai1,3.0,702997.0,76015.0,NaN,779012.0,9849922,1,oak crest institute of science,united states,910163412
39,9149624,p30,5.0,n,5p30ca196521-02,nci:1047754\,research centers,2016,cancer,NaT,NaT,abstracting; accounting; address; administrato...,nci,2.0,246631.0,801123.0,NaN,1047754.0,1864088,1,icahn school of medicine at mount sinai,united states,100296574
40,9150630,p2c,5.0,n,5p2chd050924-12,nichd:76000\,other research-related,2016,basic behavioral and social science; behaviora...,NaT,NaT,address; adolescent; adult; awareness; base; b...,zhd1,12.0,50000.0,26000.0,NaN,76000.0,1963661,1,univ of north carolina chapel hill,united states,275990001


## Cost of grants (funds)
There are 4 cost columns. Indirect and direct costs sum to total costs or to total subproject costs. Drop indirect and direct costs columns and combine the total costs into one column (costs are either listed as total cost or total subproject cost).

In [259]:
grants_2016 = grants_2016.drop(['direct_cost_amt', 'indirect_cost_amt'], axis = 1)
grants_2016['total_cost'].fillna(grants_2016['total_cost_sub_project'], inplace = True)
del grants_2016['total_cost_sub_project']

In [260]:
grants_2016.rename(columns = {'total_cost':'funds'}, inplace = True)
grants_2016.head()

,application_id,activity,application_type,arra_funded,full_project_num,funding_ics,funding_mechanism,fy,nih_spending_cats,project_start,project_end,project_terms,study_section,support_year,funds,pi_ids,num_pis,org_name,org_country,org_zipcode
0,9115627,k23,4.0,n,4k23gm104401-04,nigms:194460\,other research-related,2016,atherosclerosis; biotechnology; cardiovascular...,2013-08-01,2017-07-31,abcb1 gene; accounting; acute coronary syndrom...,ghd,4.0,194460.0,10799126,1,icahn school of medicine at mount sinai,united states,100296574
1,9128072,r01,4.0,n,4r01ns085165-04,ninds:335781\,non-sbir/sttr rpgs,2016,biotechnology; complementary and alternative m...,2013-09-30,2018-07-31,acute; analog; antioxidants; attenuation; bind...,nomd,4.0,335781.0,7017365,1,university of maryland baltimore,united states,212011508
2,9056435,d43,4.0,n,4d43tw009310-05,fic:146822\nida:75000\,other research-related,2016,aging; behavioral and social science; brain di...,2012-07-09,2017-02-28,21 year old; academy; addiction; address; alco...,zrg1,5.0,110911.0,7354078,2,university of michigan,united states,481091276
3,9056435,d43,4.0,n,4d43tw009310-05,fic:146822\nida:75000\,other research-related,2016,aging; behavioral and social science; brain di...,2012-07-09,2017-02-28,21 year old; academy; addiction; address; alco...,zrg1,5.0,110911.0,1862210,2,university of michigan,united states,481091276
4,9213716,r01,1.0,n,1r01mh111502-01,nimh:1593922\,non-sbir/sttr rpgs,2016,basic behavioral and social science; behaviora...,2016-09-09,2019-05-31,abstracting; accounting; address; aggressive b...,zmh1,1.0,531307.0,2275890,3,harvard university,united states,021385369


Save dataframe to csv

In [261]:
grants_2016.to_csv('for_analysis.csv', index = False, compression = 'gzip')

# Initial data exploration

In [262]:
grants_analysis = pd.read_csv('for_analysis.csv', compression = 'gzip')
grants_analysis.info()
grants_analysis.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84591 entries, 0 to 84590
Data columns (total 20 columns):
application_id       84591 non-null int64
activity             84574 non-null object
application_type     82964 non-null float64
arra_funded          84574 non-null object
full_project_num     84591 non-null object
funding_ics          84436 non-null object
funding_mechanism    84558 non-null object
fy                   84591 non-null int64
nih_spending_cats    73811 non-null object
project_start        70517 non-null object
project_end          70780 non-null object
project_terms        81585 non-null object
study_section        78863 non-null object
support_year         82964 non-null float64
funds                82464 non-null float64
pi_ids               84358 non-null float64
num_pis              84591 non-null int64
org_name             84275 non-null object
org_country          84338 non-null object
org_zipcode          84165 non-null object
dtypes: float64(4), int64(3), 

(84591, 20)

Checking terms

In [263]:
pd.set_option('display.max_colwidth', 500)
col_info.ix[col_info['column_name'].str.contains('activity')]
pd.set_option('display.max_colwidth', 50)

,column_name,descriptions
1,activity,"A 3-character code identifying the grant, contract, or intramural activity through which a project is supported. Within each funding mechanism , NIH uses 3-character activity codes (e.g., F32, K08, P01, R01, T32, etc.) to differentiate the wide variety of research-related programs NIH supports. A comprehensive list of activity codes for grants and cooperative agreements may be found on the Types of Grant Programs Web page. RePORTER also includes R&D contracts (activity codes begi..."


First investigate NaNs in dataset.

In [264]:
grants_analysis.isnull().sum()

application_id           0
activity                17
application_type      1627
arra_funded             17
full_project_num         0
funding_ics            155
funding_mechanism       33
fy                       0
nih_spending_cats    10780
project_start        14074
project_end          13811
project_terms         3006
study_section         5728
support_year          1627
funds                 2127
pi_ids                 233
num_pis                  0
org_name               316
org_country            253
org_zipcode            426
dtype: int64

Most nulls will not affect the ability to identify who the grant was awarded to. The identification columns are application_id (no nulls), pi_ids, org_name, org_country and org_zipcode. org_zipcode has the most nulls, but this is because several countries did not list a zipcode. The second largest column of the identiers is org_name.

In [265]:
org_null = grants_analysis.ix[grants_analysis['org_name'].isnull()]
org_null['funding_mechanism'].value_counts()

non sbir/sttr contracts    148
interagency agreements     124
intramural research         23
sbir/sttr contracts          4
Name: funding_mechanism, dtype: int64

Non-US countries awarded with NIH grants.

In [266]:
grants_analysis.ix[grants_analysis['org_country'] != 'united states'].org_country.value_counts()

south africa      155
canada            154
united kingdom     90
uganda             66
switzerland        34
kenya              34
ethiopia           33
australia          32
nigeria            30
brazil             28
peru               27
india              27
germany            27
botswana           25
zambia             22
tanzania u rep     20
mozambique         19
malawi             19
thailand           19
france             19
china              17
ghana              15
mali               14
zimbabwe           12
argentina          12
cote d'ivoire      11
haiti              10
netherlands         9
colombia            9
sweden              8
israel              8
vietnam             8
bangladesh          7
cameroon            7
philippines         6
dominican rep       6
ukraine             5
namibia             5
mexico              4
rwanda              4
cambodia            4
guatemala           4
pakistan            3
spain               3
congo dem rep       3
ireland   

In [267]:
def to_string(df, col_list):
    for col in col_list:
        df[col] = df[col].astype(str)
    return df

cols = ['application_id', 'application_type', 'fy', 'support_year', 'pi_ids']
grants_analysis = to_string(grants_analysis, col_list = cols)

In [268]:
print('The total amount of funding awarded by the NIH in 2016 was $' + \
      '{0:,}'.format(grants_analysis['funds'].sum()) + ' for ' \
     + '{0:,}'.format(len(grants_analysis)) + ' total grants.')

print('Approximately $' + '{0:,}'.format(round(grants_analysis['funds'].sum() / len(grants_analysis))) + \
      ' were awarded per grant on average.')

print('{0:,}'.format(grants_analysis['pi_ids'].value_counts().count()) + ' unique PIs and ' + \
      str(grants_analysis['org_name'].value_counts().count()) + ' unique organizations were listed.')

The total amount of funding awarded by the NIH in 2016 was $35,261,559,762.0 for 84,591 total grants.
Approximately $416,848.0 were awarded per grant on average.
48,753 unique PIs and 3210 unique organizations were listed.


In [269]:
pd.set_option('display.max_rows', 500)
by_activity = grants_analysis.groupby('activity')
cost_activity = by_activity.sum().sort_values('funds', ascending = False)

In [270]:
s = pd.Series(by_activity.size(), name = 'num_grants')
cost_activity = pd.concat([cost_activity, s], axis = 1)
cost_activity['funds_per_grant'] = cost_activity['funds'] / cost_activity['num_grants']
cost_activity.head()

,funds,num_pis,num_grants,funds_per_grant
d43,36571074.0,341,193,1.894874e+05
d71,348784.0,29,15,2.325227e+04
dp1,67354076.0,81,81,8.315318e+05
dp2,134316415.0,68,68,1.975241e+06
dp3,39468693.0,54,28,1.409596e+06


In [271]:
cost_activity.sort_values('funds_per_grant', ascending = False)

,funds,num_pis,num_grants,funds_per_grant
zih,3.631880e+08,15,15,2.421254e+07
u66,8.948906e+07,4,4,2.237227e+07
zif,4.306412e+08,24,24,1.794338e+07
zii,5.590706e+07,7,7,7.986723e+06
zib,6.138472e+07,9,9,6.820525e+06
zig,1.108553e+08,20,20,5.542766e+06
p51,7.579762e+07,15,15,5.053175e+06
zid,2.261787e+08,46,46,4.916928e+06
zik,2.799818e+07,6,6,4.666363e+06
uc7,4.600000e+07,14,14,3.285714e+06


In [274]:
grants_analysis['num_pis'].value_counts()

1     62183
2     15112
3      4458
4      1632
5       515
8       200
6       198
7       105
12       96
9        81
11       11
Name: num_pis, dtype: int64

In [272]:
#geospatial data: grants per region
#Questions for analysis: how many grants are solo vs. joint? how many PIs have solo vs. joint grants?
#How many grants per organization?
#Total costs, avg cost per PI, institution